In [48]:
import io

In [49]:
import json

In [50]:
with open('ringmanual.json', 'rt') as f_in:
    titles = json.load(f_in)

In [51]:
print(titles)

[{'title': 'E-SCOOTER', 'description': 'E-SCOOTER\nUSER MANUAL\nORIGINAL INSTRUCTIONS\n'}, {'title': 'USER MANUAL', 'description': 'E-SCOOTER\nUSER MANUAL\nORIGINAL INSTRUCTIONS\n'}, {'title': 'Contents', 'description': 'Contents\n1\n2\n3\n4\n5\n6\n7\n8\n9\n10\n11\n12\n13\n14\n15\n16\n17\nSafety Instructions \nPacking List \nProduct Function Diagram\nControl Panel/Dashboard\nAssembling Your KickScooter\nFirst Ride\nCharging\nLearning to Ride \nSecurity Tips\nFolding and Carrying\nSpeciﬁcations\nMaintenance\nBefore Your Ride\nSafety Alerts & Error Messages \nTrademarks and Legal Notices\nCertiﬁcations\nContact\n01\n04\n05\n06\n07\n08\n09\n10\n11\n13\n14\n16\n17\n18\n19\n19\n20\n'}, {'title': '', 'description': 'Contents\n1\n2\n3\n4\n5\n6\n7\n8\n9\n10\n11\n12\n13\n14\n15\n16\n17\nSafety Instructions \nPacking List \nProduct Function Diagram\nControl Panel/Dashboard\nAssembling Your KickScooter\nFirst Ride\nCharging\nLearning to Ride \nSecurity Tips\nFolding and Carrying\nSpeciﬁcations\nM

In [52]:
from openai import OpenAI

In [53]:
from openai import OpenAI

client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

In [54]:
from elasticsearch import Elasticsearch

In [55]:
es_client = Elasticsearch('http://localhost:9200') 

In [56]:
es_client.info()

ObjectApiResponse({'name': 'a1b506278180', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'Z_IPrBQ8R7a1tEpsWIT6VA', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [57]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "description": {"type": "text"},
            "title": {"type": "keyword"} 
        }
    }
}

index_name = "usermanual-answers1" #database name maybe ?

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'usermanual-answers1'})

In [58]:
from tqdm.auto import tqdm

In [59]:

for title in tqdm(titles):
    es_client.index(index=index_name,document=title) # final init of elastiserach

  0%|          | 0/21 [00:00<?, ?it/s]

In [60]:
query = 'Do I need to wear helmet while driving scooter  and can you tell me can I get hurt if I dont wear helmet?'
#query = 'Do I need to wear helmet while driving scooter ?'

In [61]:
def elastic_search(query):
    serach_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["description^3"],
                        "type": "best_fields"
                    }
                }
               
            }
        }
    }
    response = es_client.search(index=index_name,body=serach_query)
    
    response['hits']['hits']
    
    result_doc = []
    
    for hit in response['hits']['hits']:  
        result_doc.append(hit['_source'])
    return result_doc

In [62]:
elastic_search(query)

[]

In [63]:
def build_prompt(query,search_results):
    prompt_template =""" 
You are user manuel interpreter. Answer the QUESTION base on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.If CONTEXT doesn't contain  the answer output NONE


QUESTION: {question}

CONTEXT: {context}
""".strip()

    context  = ""
    for doc in search_results:
        context = context + f"title: {doc['title']}\ndescirption: {doc['description']}\n\n"
    
    prompt = prompt_template.format(question=query,context= context).strip()
    return prompt

def llm(prompt):
    response = client.chat.completions.create(
        model='gemma2:2b',
        #model='phi3',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [64]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)

    return answer

In [65]:
print(rag(query))

This text looks like a condensed user manual for an electric scooter, possibly "Inmotion" brand. Here's a breakdown of the content and some insights:

**Before You Ride (Safety & Handling)**

* **Battery Care:** This section is crucial. It details how to handle and charge the battery safely: 
    * Never touch battery terminals - this can cause severe electric shocks, fires, or damage.
    * Avoid throwing the battery in fire. It could lead to explosions, overheating, etc.
    * Do not attempt to disassemble the battery without proper authorization. Users cannot repair the battery because the design is intended solely for transportation by the customer. 
    *  Charge regularly after each use - avoid completely draining it to extend battery life, especially for long-term storage.
    * Don't leave or dispose of batteries in a fire as this could cause severe damage to them and put off dangerous fumes. Follow local regulations.

* **Environmental Responsibility:** The manual urges users 